ils en ont 92 features pour entraîner groupe 52

## Setup

In [34]:

# Import libraries for data manipulation
import pandas as pd
from summarytools import dfSummary

import numpy as np

from icecream import ic
#ic(my_var) : fait un joli print direct

from ydata_profiling import ProfileReport

# import latexify

# Import libraries for data visualization
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats("svg")

plt.style.use('seaborn-v0_8-whitegrid')

import seaborn as sns

import copy as cp

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer

from statsmodels.graphics.gofplots import ProbPlot

# Import libraries for building linear regression model
from statsmodels.formula.api import ols

# Import the required function
from scipy.stats import pearsonr

from sklearn.preprocessing import OrdinalEncoder


import statsmodels.api as sm

from sklearn.linear_model import LinearRegression

# Import library for preparing data
from sklearn.model_selection import train_test_split

# Import library for data preprocessing
from sklearn.preprocessing import MinMaxScaler

from statsmodels.stats.outliers_influence import variance_inflation_factor

from statsmodels.stats.diagnostic import het_white
from statsmodels.compat import lzip
import statsmodels.stats.api as sms

# Plot q-q plot of residuals
import pylab
import scipy.stats as stats

## R2
from sklearn.metrics import r2_score


from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")


####################
# Fonctions : 



def corr_plot(df, upper_tri=None, threshold=None):
    
    plt.figure(figsize = (12, 8))
    cmap = sns.diverging_palette(230, 20, as_cmap = True)

    corr = df.corr()
    mask = None

    if threshold != None:
        corr = corr.mask(np.abs(corr) < threshold, np.nan)

    if upper_tri:
        mask = np.tril(np.ones_like(corr, dtype=bool)) # affichera que le triangle supérieur

    sns.heatmap(corr, mask=mask, annot = True, fmt = '.2f', cmap = cmap)
    plt.show()

    return corr

# RMSE
# @latexify.function
def rmse(predictions, targets):
    return np.sqrt(((targets - predictions) ** 2).mean())

# MAPE
# @latexify.function
def mape(predictions, targets):
    return np.mean(np.abs((targets - predictions)) / targets) * 100

# MAE
# @latexify.function
def mae(predictions, targets):
    return np.mean(np.abs((targets - predictions)))

# Function to check VIF
def checking_vif(train):
    vif = pd.DataFrame()
    vif["feature"] = train.columns

    # Calculating VIF for each feature
    vif["VIF"] = [
        variance_inflation_factor(train.values, i) for i in range(len(train.columns))
    ]
    return vif


In [2]:
df = pd.read_csv("./data/X_train_Hi5.csv")
df = df.sample(n=10_000, random_state=42)
df.head()

,row_index,piezo_station_department_code,piezo_station_update_date,piezo_station_investigation_depth,piezo_station_department_name,piezo_station_commune_code_insee,piezo_station_pe_label,piezo_station_bdlisa_codes,piezo_station_altitude,piezo_station_bss_code,...,prelev_longitude_2,prelev_latitude_2,prelev_commune_code_insee_2,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_med_living_level,insee_%_ind,insee_%_const,piezo_groundwater_level_category
2613083,3067028,33,Fri Jun 28 07:31:38 CEST 2024,123.0,Gironde,33517,LE PORT (SOUSSANS-33),['334AG04'],3.59,07795X0106/PZEM9,...,NaN,NaN,NaN,NaN,19,1692.0,23100,6.3,18.5,Average
1268326,1334103,49,Fri Jan 12 10:28:05 CET 2024,93.5,Maine-et-Loire,49194,FONTAINE-MILONJurassique,['139AP14'],40.00,04552X0111/PZ2,...,-0.281423,47.471283,49194.0,18620902.0,8.8,5815.0,23560,8.8,12.5,Average
234393,246594,27,Mon Sep 23 19:38:51 CEST 2024,12.0,Eure,27391,PUITS DE L'ANCIENNE GARE (MARCILLY-SUR-EURE),['121AS01'],75.00,01807X0051/S1,...,1.365070,48.713810,28404.0,29117529.0,1.3,1588.0,25340,16.6,15.9,High
1670802,1756732,44,Fri Jun 28 07:31:38 CEST 2024,8.0,Loire-Atlantique,44025,Forage de LA RIVIERE,['104AB01'],5.00,04503X0068/PZ2,...,-1.881925,47.275110,44045.0,8572341.0,1,4005.0,23620,18.6,19.8,Average
2804886,3267483,77,Thu Sep 19 14:26:45 CEST 2024,15.0,Seine-et-Marne,77341,PIÉZOMÈTRE DE NOYEN-SUR-SEINE (NOYEN-SUR-SEINE...,['928AC01'],57.50,02606X0112/G5,...,NaN,NaN,NaN,NaN,0,384.0,24270,0,8.3,Very Low


In [43]:
target = "piezo_groundwater_level_category"

## Preprocessing

In [54]:
# On tolère 19% de nan dans les colonnes
for proportion_nan_prct in [19]:
    new_df = cp.deepcopy(df)
    for col in new_df.columns:
        if new_df[col].isnull().sum() * 100 / len(df) > proportion_nan_prct:
            new_df.drop(col, axis=1, inplace=True)

# new_df.info()

# Gérer les duplicatas
new_df.drop_duplicates(inplace=True)

# Drop les features indépendantes qui sont ultra corrélées entre elles (id...)
features_to_drop = ["piezo_station_commune_code_insee",
                    "piezo_station_pe_label",
                    "piezo_station_bdlisa_codes",
                    "piezo_station_bss_code",
                    "piezo_station_commune_name",
                    "piezo_station_bss_id",
                    "piezo_bss_code",
                    "piezo_station_update_date",
                    "piezo_qualification",
                    "piezo_continuity_code",
                    "piezo_continuity_name",
                    "piezo_producer_name",
                    "piezo_measure_nature_name",
                    "meteo_name",
                    "hydro_station_code",
                    "hydro_method_code",
                    "hydro_method_label",
                    "insee_med_living_level",
                    "meteo_id",
                    "hydro_qualification_label",
                    "hydro_status_code",
                    "piezo_station_department_name"]

features = new_df.drop(features_to_drop, axis=1)
# features.info()

num_col_features =      ["piezo_station_investigation_depth", 
                        "piezo_station_altitude", 
                        "piezo_station_longitude", 
                        "piezo_station_latitude", 
                        "piezo_producer_code", 
                        "meteo_latitude", 
                        "meteo_longitude", 
                        "meteo_altitude", 
                        "meteo_rain_height", 
                        "meteo_temperature_min", 
                        "meteo_time_tn", 
                        "meteo_temperature_max", 
                        "meteo_time_tx", 
                        "meteo_temperature_avg", 
                        "meteo_temperature_avg_threshold",
                        "meteo_frost_duration", 
                        "meteo_amplitude_tn_tx", 
                        "meteo_temperature_avg_tntm", 
                        "meteo_evapotranspiration_grid", 
                        "distance_piezo_meteo", 
                        "hydro_observation_result_elab", 
                        "hydro_longitude",
                        "hydro_latitude", 
                        "distance_piezo_hydro", 
                        "prelev_other_volume_sum", 
                        "insee_%_agri", 
                        "insee_pop_commune", 
                        "insee_%_ind", 
                        "insee_%_const"]

cat_col_features = ["piezo_station_department_code",  
                    "piezo_obtention_mode", 
                    "piezo_status", 
                    "piezo_measure_nature_code", 
                    "hydro_status_label", 
                    "hydro_qualification_code",  
                    "hydro_hydro_quantity_elab"]


# On traite le dataframe features entier
new_features = cp.deepcopy(features)

new_features[num_col_features] = new_features[num_col_features].replace('N/A - division par 0', np.nan)
new_features[num_col_features] = new_features[num_col_features].replace('nan', np.nan)

for col in num_col_features:
    new_features[col].astype("float")

# imputation des valeurs numériques manquantes (np.nan) : 

imputer_mean_posterior = IterativeImputer(random_state=42, sample_posterior=True)

imputer_mean_posterior.fit(new_features[num_col_features])
values_imputed = imputer_mean_posterior.transform(new_features[num_col_features])

new_features[num_col_features] = values_imputed

# # vérif plus de nan en numérique
# nan_count = new_features[num_col_features].isna().sum()
# print(nan_count)
# total_nan_count = new_features[num_col_features].isna().sum().sum()
# print(total_nan_count)

# Gérer les 3 features datetime

new_features['DATE_piezo_measurement_date'] = pd.to_datetime(new_features['piezo_measurement_date'])

new_features['year_piezo_measurement'] = new_features['DATE_piezo_measurement_date'].dt.year
new_features['month_piezo_measurement'] = new_features['DATE_piezo_measurement_date'].dt.month
new_features['day_piezo_measurement'] = new_features['DATE_piezo_measurement_date'].dt.day
new_features = new_features.drop(['piezo_measurement_date'], axis=1)

new_features['DATE_meteo_date'] = pd.to_datetime(new_features['meteo_date'])

new_features['year_meteo_date'] = new_features['DATE_meteo_date'].dt.year
new_features['month_meteo_date'] = new_features['DATE_meteo_date'].dt.month
new_features['day_meteo_date'] = new_features['DATE_meteo_date'].dt.day
new_features = new_features.drop(['meteo_date'], axis=1)


new_features['DATE_hydro_observation_date_elab'] = pd.to_datetime(new_features['hydro_observation_date_elab'])

new_features['year_hydro_observation_date_elab'] = new_features['DATE_hydro_observation_date_elab'].dt.year
new_features['month_hydro_observation_date_elab'] = new_features['DATE_hydro_observation_date_elab'].dt.month
new_features['day_hydro_observation_date_elab'] = new_features['DATE_hydro_observation_date_elab'].dt.day
new_features = new_features.drop(['hydro_observation_date_elab'], axis=1)


new_features_2 = cp.deepcopy(new_features) # au cas où je fais un truc nul


# target_feature = new_features_2[target]

# Ordinal encoding :

ord_cols = ["piezo_obtention_mode", 
            "piezo_status", 
            "piezo_measure_nature_code", 
            "hydro_status_label",
            "hydro_qualification_code",
            "piezo_groundwater_level_category"]

for col in ord_cols:

    temp = new_features_2[col]
    temp = pd.DataFrame(temp)

    ordinal_encoder = OrdinalEncoder()
    ordinal_encoder.fit(temp)
    ordinal_enc = ordinal_encoder.transform(temp)

    new_features_2['ORDINAL_' + col] = ordinal_enc


new_features_2 = new_features_2.drop(ord_cols, axis=1)

# test.sample(50)

# Onehot encoding
onehot_cols = ["piezo_station_department_code", "hydro_hydro_quantity_elab"]
new_features_2 = pd.get_dummies(new_features_2, columns=onehot_cols, drop_first=True)
# new_features_2[target] = target_feature # tout est onehot encodé sauf la target var

# new_features_2.info()

In [55]:
pd.set_option('display.max_columns', None)
new_features_2.head()

,row_index,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,piezo_producer_code,meteo_latitude,meteo_longitude,meteo_altitude,meteo_rain_height,meteo_temperature_min,meteo_time_tn,meteo_temperature_max,meteo_time_tx,meteo_temperature_avg,meteo_temperature_avg_threshold,meteo_frost_duration,meteo_amplitude_tn_tx,meteo_temperature_avg_tntm,meteo_evapotranspiration_grid,distance_piezo_meteo,hydro_observation_result_elab,hydro_longitude,hydro_latitude,distance_piezo_hydro,prelev_other_volume_sum,insee_%_agri,insee_pop_commune,insee_%_ind,insee_%_const,DATE_piezo_measurement_date,year_piezo_measurement,month_piezo_measurement,day_piezo_measurement,DATE_meteo_date,year_meteo_date,month_meteo_date,day_meteo_date,DATE_hydro_observation_date_elab,year_hydro_observation_date_elab,month_hydro_observation_date_elab,day_hydro_observation_date_elab,ORDINAL_piezo_obtention_mode,ORDINAL_piezo_status,ORDINAL_piezo_measure_nature_code,ORDINAL_hydro_status_label,ORDINAL_hydro_qualification_code,ORDINAL_piezo_groundwater_level_category,piezo_station_department_code_02,piezo_station_department_code_03,piezo_station_department_code_04,piezo_station_department_code_05,piezo_station_department_code_06,piezo_station_department_code_07,piezo_station_department_code_08,piezo_station_department_code_09,piezo_station_department_code_10,piezo_station_department_code_11,piezo_station_department_code_12,piezo_station_department_code_13,piezo_station_department_code_14,piezo_station_department_code_15,piezo_station_department_code_16,piezo_station_department_code_17,piezo_station_department_code_18,piezo_station_department_code_19,piezo_station_department_code_21,piezo_station_department_code_22,piezo_station_department_code_23,piezo_station_department_code_24,piezo_station_department_code_25,piezo_station_department_code_26,piezo_station_department_code_27,piezo_station_department_code_28,piezo_station_department_code_29,piezo_station_department_code_2A,piezo_station_department_code_2B,piezo_station_department_code_30,piezo_station_department_code_31,piezo_station_department_code_32,piezo_station_department_code_33,piezo_station_department_code_34,piezo_station_department_code_35,piezo_station_department_code_36,piezo_station_department_code_37,piezo_station_department_code_38,piezo_station_department_code_39,piezo_station_department_code_40,piezo_station_department_code_41,piezo_station_department_code_42,piezo_station_department_code_43,piezo_station_department_code_44,piezo_station_department_code_45,piezo_station_department_code_46,piezo_station_department_code_47,piezo_station_department_code_48,piezo_station_department_code_49,piezo_station_department_code_50,piezo_station_department_code_51,piezo_station_department_code_52,piezo_station_department_code_53,piezo_station_department_code_54,piezo_station_department_code_55,piezo_station_department_code_56,piezo_station_department_code_57,piezo_station_department_code_58,piezo_station_department_code_59,piezo_station_department_code_60,piezo_station_department_code_61,piezo_station_department_code_62,piezo_station_department_code_63,piezo_station_department_code_64,piezo_station_department_code_65,piezo_station_department_code_66,piezo_station_department_code_67,piezo_station_department_code_68,piezo_station_department_code_69,piezo_station_department_code_70,piezo_station_department_code_71,piezo_station_department_code_72,piezo_station_department_code_73,piezo_station_department_code_74,piezo_station_department_code_75,piezo_station_department_code_76,piezo_station_department_code_77,piezo_station_department_code_78,piezo_station_department_code_79,piezo_station_department_code_80,piezo_station_department_code_81,piezo_station_department_code_82,piezo_station_department_code_83,piezo_station_department_code_84,piezo_station_department_code_85,piezo_station_department_code_86,piezo_station_department_code_87,piezo_station_department_code_88,piezo_sta

In [56]:
new_features_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 2613083 to 1625530
Columns: 142 entries, row_index to hydro_hydro_quantity_elab_QmM
dtypes: bool(94), datetime64[ns](3), float64(35), int32(9), int64(1)
memory usage: 4.5 MB


### Br

#### Tout doit être en numérique après encoding (pas d'object)

In [60]:
dfSummary(new_features_2)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,row_index[int64],Mean (sd) : 1565938.2 (960634.6)min < med < max:83.0 < 1495360.5 < 3294076.0IQR (CV) : 1493751.2 (1.6),"10,000 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/OQEPoAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADFUlEQVR4nO3asU/bQBSA8Xc4QZeE2gqkyealIxKDJRaWbh37x3btBhJInTN06ZQFIUUmUQxWLk46VB2QqhCfbchLv99+8Q3f3bMRZrPZCLDvjt57A8AuWu+9AbxkjOmIyHGFn1huNpvnuvazLwh1jxhjOmdnZ1/DMOz7/sZ8Pk+NMd8OLVZC3S/HYRj2r66unqMoyssuns1m9vb2tj+dTo9F5M1DrTgNtk4CQt1DURTlw+HwyXN5p9bN7KjqNHhtEjQaKu9b/xXvabDLJHg11Aqx2dPT0y9RFPU81oqIyOPj48IY811ESo9B+fMXjbXvs4VD4qXCNNg6CbaGWuU6d85ZETm/vLz8MRgM5mXX39/ff7i7u/scx/EgCIJSoRZF0c6y7NPJycmvo6MjV/bZItU+Sioc7nC9Xrc91h28125U7+t8Mpn0x+Nx11rrfE5YmqYda203SZJ8NBqlHs/+mCTJz7JrRf6Mopubm+F0Oh0YY8oeMu9J4pyzRVGc53l+LSK+76gHaad3VJ/rPE3TWl7qe72e97N91oqIZFnWds5dxHEclL3Nq0ySv4d7tVoFpTZck4rfFI1OA776/2G5XLYq3uZek6Suw+2j6ld709OAULeocpsrVOlvuE1PA0LFC75f7U0fUP4pBSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlRovfcGUK+iKNoiEhpjyi4N1+t1u4Et1YJQD0iWZW3n3EUcx0EQBHmZtc45WxTFeZ7n1yLy1NAWvRHqAVkuly1rbTdJknw0GqVl1k4mk/54PO6uVqugqf1VQagHqNfr5cPhsNStmKZpp6n91IGPKaiw0406m81s2R9eLBZWRCTLMvvw8NB9y/Van61131XX79LXbyJpcK7HwEbbAAAAAElFTkSuQmCC"">",0(0.0%)
2,piezo_station_investigation_depth[float64],Mean (sd) : 89.2 (256.0)min < med < max:-768.6 < 40.0 < 5233.4IQR (CV) : 68.9 (0.3),"1,594 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/OQEPoAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAB10lEQVR4nO3XT07CYBCG8RkRbI35SlNlyZaFiQfgEB7WQ7Bww0UohCZYsCmfJ/APFcRXn99+prN40qYeYzTgt7s49wHAV1yecrm7p2Y2+MaK1xhjfax7oOtkobp7WhTFYwgh77qjqqqVuz8RK075Rh2EEPLpdFpnWbY9dHi9Xiez2Swvy3JgZoT6z530029mlmXZdjQavXQcT496DGTxMwUJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJhAoJl+c+4CNt2/bNLLh7l/HXGGN95JNwJp+G6u6pmQ067A77/b7fYc7MzDabTb9pmofxeNzr9XrbQ+erqlq5+xOx/g0fhuruaVEUjyGE/NDFTdMkbdveLxaL5y6HLZfLmyRJrieTiQ+Hw90hs3VdX83n87uyLG/dveryfPy4XYzx3RfSGzxEWGmtiQZmAAAAAElFTkSuQmCC"">",0(0.0%)
3,piezo_station_altitude[float64],Mean (sd) : 148.1 (197.4)min < med < max:-999.0 < 107.0 < 2150.0IQR (CV) : 146.0 (0.8),"1,480 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/OQEPoAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACGElEQVR4nO3Yu47aUBRG4b25jRmILYRkOpr0KSh5iDzstOl5i/QIyQJbeCAgc1Klm1xsPIGfWV+/j3exJOscDyEYcO86t14A+Be9Wy/wXtx9aGaDK444hRAObe2D6zxkqO4+nE6nX+M4njQ9oyiKrbu/EOt9eMhQzWwQx/FkuVwekiQ51h3O8zxarVaTLMsGZkaod+BRQzUzsyRJjmmavjYcH7a6DK7CZQoSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSCBUSerde4F5VVdU3s9jdm4yfQgiHllf60Aj1DWVZ9s/n85f5fN7tdrvHuvNFUWzd/YVY20OobzidTr0oip4Xi8VxNptt68zmeR6tVqtJlmUDMyPUlhDqH4xGo2Oapq8NRoetL/PBcZmCBEKFBH797+DKF4OOmV2u+PxDvjj8NVR3j8zs6T/s0qb4crn08jyPmgzv9/vIzKwsy2iz2TzXmV2v15+avhhUVdUvy/LzeDz+3ul0znVmf9ntdnt3/2ZmtV8rbuxHCOG3O/8EI0t6eZwCdHgAAAAASUVORK5CYII="">",0(0.0%)
4,piezo_station_longitude[float64],Mean (sd) : 2.2 (2.7)min < med < max:-4.7 < 2.0 < 9.5IQR (CV) : 4.3 (0.8),"2,493 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/OQEPoAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACx0lEQVR4nO3dQW7iMBSA4

#### ordinal encoding

In [17]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
ord_cols = ["piezo_obtention_mode", "piezo_status"]
test = new_features_2[ord_cols]
test = pd.DataFrame(test)
test.head()

for col in ord_cols:

    temp = test[col]
    temp = pd.DataFrame(temp)

    ordinal_encoder = OrdinalEncoder()
    ordinal_encoder.fit(temp)
    ordinal_enc = ordinal_encoder.transform(temp)

    test['NUM_' + col] = ordinal_enc

test.sample(50)

ordinal encoding : 

piezo_obtention_mode
1. Valeur mesurée
2. Mode d'obtention inconnu
3. Valeur reconstituée

piezo_status
1. Donnée contrôlée niveau 2
2. Donnée contrôlée niveau 1
3. Donnée brute
4. Donnée interprétée

piezo_measure_nature_code
1. 0
2. N
3. nan
4. I
5. D
6. S

hydro_status_label
1. Donnée validée
2. Donnée pré-validée
3. Donnée brute
4. Donnée corrigée

hydro_qualification_code
1. 20
2. 12
3. 16




one hot : 
piezo_station_department_code
hydro_hydro_quantity_elab


In [11]:
dfSummary(new_features[cat_col_features])

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,piezo_station_department_code[object],1. 332. 173. 344. 405. 496. 247. 688. 769. 1310. 8011. other,"577 (5.8%)513 (5.1%)441 (4.4%)393 (3.9%)386 (3.9%)295 (2.9%)250 (2.5%)221 (2.2%)214 (2.1%)205 (2.1%)6,505 (65.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/OQEPoAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAEOElEQVR4nO3dTW5aSQBG0SJCVjIBWZZYDIvoxfYi2I1lwSCOMqEH+VEcA04671276XNmkRkg5aoQT/XZi+PxOKDw7rXfAP8fYiMjNjJiIyM2MmIjIzYyYiMjNjKLMcaHMcbNC6/7fDweH4P3wxVb3t3d/bVarW4vvehwODwsFou/BcefWK5Wq9vtdvu4Xq8/nXrBfr9/v9vtbu/v72/GGGLjX1uOMcZ6vf602Ww+Xnjdh+j9cMV8QSAjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjLLMb7cWTv3gks/g9+xPBwOD7vd7nZcuLN2OBwexhifu7fFNbJBILPwK7Oo/MrJ5lRjEi+uqyyrmMrFdZVlFVP6lXWVZRWT8FCXjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNzMXBi7ELU3px8GLswlRcCydj8ELmpZPNqcZkLg5ejF2Y0tnBi7ELU3tp8GLswmQ81CUjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiNzdvBi7MLULg5ejF2YkmvhZAxeyPiCQObSx6iPUCZ1dl1lWcXUTq6rLKuYw6V1lWUVk/IFgYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjczJdZVlFXM4u66yrGJqroWTsa4i42QjY/BCxuCFjMELGQ91yYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kDF4IeNaOBmDFzLnTjanGpM7OXgxdmEOzwYvxi7M5dzgxdiFyXmoS0ZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZJ6tqyyrmMvJdZVlFXOwQSBjXUXm1MnmVGMWz9ZVllXM5cm6yrKKOZ1aV1lWMQsPdcmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2Mg8GbwYuzCnZ4MXYxfm4lo4GYMXMj+fbE41ZvNk8GLswpzefRu8bLfbx6/RnfpzkPDHvg9evv7b2IXZeKhLRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRG5tlfeIG5/Dh4MXZhVq6FkzF4IeMLAhkfo2Ssq8hYV5GxriLjCwIZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVkXAsnY11FxslGxuCFjMELGYMXMh7qkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkTF4IWPwQsa1cDIGL2R+Ptl4G67yE+bJ4IW34VqHR8tvg5cfdgi8ov1+/363293e39/fjDGuK7YxvgxeNpvNx9d+M3x3lcMjD3XJiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIGLy8Mdf8f/Hj4OUqr7X8F13r8Mi18LfpKq+FG7yQ8QWBjNjIiI2M2MiIjYzYyIiNjNjIiI3MP/UPKcX/nvVZAAAAAElFTkSuQmCC"">",0(0.0%)
2,piezo_obtention_mode[object],1. Valeur mesurée2. Mode d'obtention inconnu3. Valeur reconstituée,"9,399 (94.0%)480 (4.8%)121 (1.2%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABFCAYAAABdVZTTAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/OQEPoAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABrUlEQVR4nO3csW2DQACF4QdCTmgOIUsewwN4iAybIRjAeyDEFcFxQ4pEURrgCvOQyf/VFFf8snUWz9k4jgIc8q0PgP8jk1RKOmx9EDy9+ziOw9wDxfF4fAsh1K4TYZ9ijF2WZe9zwRUhhPpyuQxVVd2ch8N+9H3/2jRN3bbtQdJ0bJJUVdXtdDp92E6HPSqXHuCCABtigw2xwYbYYENssCE22BAbbIgNNsQGG2KDDbHBhthgQ2ywITbYFNL3+0hbHwTPK7WfIsbYNU1TK+F9JGBKjLGTdJ97hg0CHmVxg5Ax5YPL0ifbYq1Aqtl1VcpiBkg1ua5KXcwAqZbWVdxQ8TD8qAsbYoMNscGG2GBDbLAhNtgQG2yIDTbEBhtigw2xwYbYYENssCE22Eyuq1hc4dFm11UpixkgFRsE2LCugg0XBNj8/RrlKxOr+p3yMdvD2vIQQn0+n/WzHeVvGLCaXJLKsvzc+iDYPy4IsCE22BAbbIgNNsQGG2KDDbHBhthgQ2ywITbYEBtsiA02xAYbYoNNLknDMLxsfRDsXx5j7K7XK7M9rI4NAmyY8sHmC1+ylC8VDzRbAAAAAElFTkSuQmCC"">",0(0.0%)
3,piezo_status[object],1. Donnée contrôlée niveau 22. Donnée contrôlée niveau 13. Donnée brute4. Donnée interprétée,"7,534 (75.3%)2,022 (20.2%)359 (3.6%)85 (0.9%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABcCAYAAAB5jMeAAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/OQEPoAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAB+klEQVR4nO3dQUrkUBiF0T8SpJ0kFIFaTC3CxfYiajdFURm04iQ9aHsiKkgn94X2nBU88ONpQi52y7IUJNy1PgDfh9iI6arqoaruWx+EVb0sy/LU+hBv9dM0PQ7DcGh9ENYzz/O167qfewuuH4bhcDqdnsZxfG59GP7d7Xb7cT6fD5fL5b6q9hVbVdU4js/H4/FX68OwmofWB3iPBwRixEaM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI6av+vNZSuuDsI49/yz7eZ6v5/P5UDv9LIWvm+f5WlUvrc/xls/C/0+7/Cy8M+UjxQMCMa1/je7yumcbTddVe10BsY1m66o9r4DYRut1ldct34gHBGLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzENJvy7XlyxjaaTvn2OjljGzYIxJjyEfPZzebWYVUfrqssn1jbu+sqyye28Nm6yvKJVXmpS4zYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI2Yd9dVlk9s4cN1leUTa7NBIMa6ipi/N1u5xdhaP03TY1WVJRVbuxuG4fA65fMPbtmU92zEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjpn9dUVVZUrExGwRirKuI8TcbMb8BJ2W42+h8aZAAAAAASUVORK5CYII="">",0(0.0

## Data manipulation

In [58]:
# Separate the dependent variable and indepedent variables

ord_target = "ORDINAL_" + target

Y = new_features_2[ord_target]

X = new_features_2.drop(columns=ord_target)

# Add the intercept term
X = sm.add_constant(X)

# splitting the data in 70:30 ratio of train to test data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state=42)

## Feature Engineering & Selection

In [59]:
checking_vif(X_train)

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''